# Student Loan Risk with Deep Learning

In [20]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [21]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [22]:
loans_df.describe()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,0.534709
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.498950
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,0.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,0.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,1.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,1.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,1.000000


In [23]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [24]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [25]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']

# Display a sample of y
y[:5]

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [26]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.copy().drop(columns=["credit_ranking"])

# Review the features DataFrame
X

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [27]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [28]:
# Create a StandardScaler instance
std_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [29]:
# Define the the number of inputs (features) to the model
input_features = len(X.columns)

# Review the number of features
input_features

11

In [30]:
# Define the number of hidden nodes for the first hidden layer
f_layer = 117

# Define the number of hidden nodes for the second hidden layer
s_layer = 161

# Define the number of neurons in the output layer
o_layer = 1

In [31]:
# Create the Sequential model instance
nn_model = keras.models.Sequential()

# Add the first hidden layer
nn_model.add(keras.layers.Dense(units=f_layer, activation='relu', input_dim=input_features))

# Add the second hidden layer
nn_model.add(keras.layers.Dense(units=s_layer, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(keras.layers.Dense(units=o_layer, activation='sigmoid'))

/Applications/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 117)            │         1,404 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 161)            │        18,998 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           162 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,564 (80.33 KB)

 Trainable params: 20,564 (80.33 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [33]:
# Compile the Sequential model
nn_model.compile(optimizer='adam', #keras.optimizers.Adam(), 
                 loss='binary_crossentropy',#keras.losses.BinaryCrossentropy(), 
                 metrics=['accuracy'])

In [34]:
# Fit the model using 50 epochs and the training data
nn_model.fit(X_train_scaled, y_train, epochs=75, 
             validation_data=(X_test_scaled, y_test))

Epoch 1/75


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6587 - loss: 0.6269 - val_accuracy: 0.7156 - val_loss: 0.5332
Epoch 2/75
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 942us/step - accuracy: 0.7415 - loss: 0.5084 - val_accuracy: 0.7312 - val_loss: 0.5207
Epoch 3/75
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 875us/step - accuracy: 0.7589 - loss: 0.4958 - val_accuracy: 0.7500 - val_loss: 0.5043
Epoch 4/75
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - accuracy: 0.7706 - loss: 0.4867 - val_accuracy: 0.7437 - val_loss: 0.4989
Epoch 5/75
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step - accuracy: 0.7871 - loss: 0.4696 - val_accuracy: 0.7281 - val_loss: 0.4970
Epoch 6/75
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 821us/step - accuracy: 0.7685 - loss: 0.4850 - val_accuracy: 0.7406 - val_loss: 0.4891
Epoch 7/75
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 809us/step - accuracy: 0.7596 - loss: 0.4780 - val_accuracy: 0.7437 - val_loss: 0.5006
Epoch 8/75
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step - accuracy: 0.7648 - loss: 0.4846 - val_accuracy: 0.7531 - val_lo

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [35]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, acc = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Accuracy: {acc}, Loss: {loss}")

10/10 - 0s - 1ms/step - accuracy: 0.8000 - loss: 0.5767
Accuracy: 0.800000011920929, Loss: 0.5767346024513245


As we are seeing overfitting and accuracy less than 85%, lets tune model Keras Hyperband Tuner and see if we get better results.

In [36]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    f_activation = hp.Choice('activation',['relu','tanh', 'leaky_relu', 'elu'])
    h_activation = hp.Choice('activation',['relu','tanh', 'leaky_relu', 'elu'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=200,
        step=2), activation='relu', input_dim=len(X.columns)))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=200,
            step=2),
            activation=h_activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [37]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=75,
    hyperband_iterations=2)

tuner.search(X_train_scaled,y_train,epochs=75,validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 00m 05s]
val_accuracy: 0.815625011920929

Best val_accuracy So Far: 0.8374999761581421
Total elapsed time: 00h 04m 04s


In [38]:
# printing the top model parameters
best_hyper = tuner.get_best_hyperparameters()
print(best_hyper[0].values)
    
best_model = tuner.get_best_models()[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss:.5f}, Accuracy: {model_accuracy:.5f}")

{'activation': 'relu', 'first_units': 175, 'num_layers': 2, 'units_0': 105, 'units_1': 73, 'units_2': 181, 'tuner/epochs': 75, 'tuner/initial_epoch': 25, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0165'}
10/10 - 0s - 9ms/step - accuracy: 0.8375 - loss: 0.7917
Loss: 0.79172, Accuracy: 0.83750


Store the best model and reuse later to evaluate data

### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [39]:
# Set the model's file path
model_path = "model/student_loan.keras"

# Export your model to a keras file
best_model.save(model_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [40]:
# Set the model's file path

# Load the model to a new object
loaded_model = keras.models.load_model(model_path)

/Applications/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [41]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Display a sample of the predictions
predictions

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step


array([[6.50767267e-01],
       [1.19878142e-03],
       [7.90458798e-01],
       [4.91471837e-05],
       [9.99455094e-01],
       [1.33219408e-04],
       [5.59686214e-06],
       [4.90245060e-04],
       [8.71813416e-01],
       [8.72425437e-01],
       [9.99999940e-01],
       [1.86503291e-01],
       [9.97280478e-01],
       [1.13656139e-02],
       [3.85757983e-01],
       [9.92458463e-01],
       [5.04805241e-03],
       [7.92173743e-01],
       [9.99951482e-01],
       [1.62053283e-03],
       [3.90808097e-08],
       [6.22437775e-01],
       [7.34591018e-03],
       [9.99837995e-01],
       [1.11636624e-01],
       [9.76069391e-01],
       [9.99990761e-01],
       [5.26475668e-01],
       [1.73900225e-05],
       [9.98041034e-01],
       [6.62061423e-02],
       [3.10245693e-01],
       [9.60498273e-01],
       [4.70864594e-01],
       [7.20728099e-01],
       [1.16996197e-02],
       [1.00000000e+00],
       [9.91787434e-01],
       [5.72328381e-02],
       [9.99595642e-01],


In [42]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(predictions, columns=['predictions'])
predictions_df['predictions'] = round(predictions_df['predictions'], 0)
predictions_df

,predictions
0,1.0
1,0.0
2,1.0
3,0.0
4,1.0
...,...
315,1.0
316,0.0
317,0.0
318,1.0


### Step 4: Display a classification report with the y test data and predictions

In [43]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df['predictions']))

              precision    recall  f1-score   support

           0       0.81      0.83      0.82       141
           1       0.86      0.84      0.85       179

    accuracy                           0.84       320
   macro avg       0.83      0.84      0.84       320
weighted avg       0.84      0.84      0.84       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

